In [1]:
from bai import Bai
from Bio import bgzf
import io
import gzip

import pysam 

In [2]:
#block gzip end of file marker
_bgzf_eof = b"\x1f\x8b\x08\x04\x00\x00\x00\x00\x00\xff\x06\x00BC\x02\x00\x1b\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00"

In [3]:
bai ='data/testdata.bam.bai'
bam = 'data/testdata.bam'

In [4]:
#thats where the bai file magic happens :)
b_idx=Bai(bai)

In [5]:
#create header bam dict 

with pysam.AlignmentFile(bam) as b:
    headerobject = b.header

In [6]:

#ref_id = 1 #that not the chrom but the id in the bam header its like 0:chr1,1:chr2..


chromosome='chr1'
ref_id= headerobject.get_tid(chromosome)
start_coords=5_000_000
end_coords=10_000_000


#somehow i still need this padd to read in the next region to get all the read I want
padd=100_000


In [24]:
header = b_idx.query(0,0,)

In [53]:
#query header, query return the start byte of the first data block --> everything before is header
end=1
header = b_idx.query(0,0,end)

while header ==None:
    end=end+10_000
    header = b_idx.query(0,0,end)
    
header_byte,header_offset = bgzf.split_virtual_offset(header.voffset_beg)

# opens bam and read only header bytes, no alignments
with open(bam,'rb')as f:
    header_bytes  = f.read(header_byte)

In [54]:

# here I do 2 queries 1 for the start and one for the end, as a result I get 4 byte coords + 4 offsets


#start byte start coord
#end byte start coord


#start byte end coord
#end byte end coord


start = b_idx.query(ref_id,start_coords,start_coords+1)
start_startb,start_startoff = bgzf.split_virtual_offset(start.voffset_beg)
start_endb,start_endoff = bgzf.split_virtual_offset(start.voffset_end)


end = b_idx.query(ref_id,end_coords+padd,end_coords+1+padd)
end_startb,end_startoff = bgzf.split_virtual_offset(end.voffset_beg)
end_endb,end_endoff = bgzf.split_virtual_offset(end.voffset_end)

while not end_endb>end_startb:
    padd=padd+10_000
    end = b_idx.query(ref_id,end_coords+padd,end_coords+1+padd)
    end_startb,end_startoff = bgzf.split_virtual_offset(end.voffset_beg)
    end_endb,end_endoff = bgzf.split_virtual_offset(end.voffset_end)
    

In [55]:

    
with open(bam,'rb')as f:
    f.seek(start_startb)
    chunk1 = f.read(end_startb-start_startb)
    chunk2 = f.read(end_endb-end_startb)

In [56]:
len(chunk2)

17587

In [57]:
filehndl = io.BytesIO(chunk1)
values = [x for x  in bgzf.BgzfBlocks(filehndl)]
frst_blck = chunk1[:values[0][1]]
frst_blck_cln = gzip.compress(gzip.decompress(frst_blck)[start_startoff:])
blks_nofirst_nolast = chunk1[values[0][1]:]


In [58]:
len(chunk1)

104252

In [59]:
filehndl = io.BytesIO(chunk2)
values = [x for x  in bgzf.BgzfBlocks(filehndl)]
frst_blck_end = chunk2[:values[0][1]]
frst_blck_end_cln = gzip.compress(gzip.decompress(frst_blck_end)[:end_startoff])
#blks_nofirst_nolast = chunk1[values[0][1]:]

In [60]:
bam_out = 'ROI.bam'

In [61]:
with open(bam_out,'wb')as o:
    o.write(header_bytes)
    o.write(frst_blck_cln)
    o.write(blks_nofirst_nolast)
    o.write(frst_blck_end_cln)
    o.write(_bgzf_eof)